In [1]:
import glob
from music21 import converter, instrument, note, chord
notes = []
for file in glob.glob('midi_songs/*.mid'):
    midi = converter.parse(file)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)

    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()

    else: # file has no notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))

        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))




In [2]:
len(notes)

45976

In [3]:
import numpy as np
from keras.utils import np_utils

sequence_length = 100

# get all the pitch names
pitchnames = sorted(set(item for item in notes))

#Create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

#Create input sequences and the corresponding outputs

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i+sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# Convert network_input to a numpy array
network_input_array = np.array(network_input)

# Determine the number of unique elements in network_input
unique_elements = len(np.unique(network_input_array))

# Normalize input
network_input = network_input / float(unique_elements)

network_output = np_utils.to_categorical(network_output)





NameError: name 'np_utils' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
model = Sequential(
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
)

# Load the weights to each node

model.load_weights('weights.hdf5')

In [ ]:
filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5'

checkpoint = ModelCheckpoint(
    filepath, monitor = 'loss',
    verbose = 0,
    save_best_only = True,
    mode = 'min'
)

callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs = 200, batch_size = 64,
callbacks = callbacks_list)

In [ ]:
start = numpy.random.randint(0, len(network_input) - 1)

int_to_note = dict((number, note) for number , note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

# Generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose = 0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append(index)
    pattern = pattern[1: len(pattern)]

In [ ]:
offset  = 0
output_notes = []

# Create note and chord objects based on the values generated by the model
for pattern in prediciton_output:
    # Pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_notes in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # Pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5
    

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')